# Classification
## MSDS 7349 - Section 401
## Project 2

[Data Science @ Southern Methodist University](https://datascience.smu.edu/)

# Table of Contents
* [Team Members](#Team-Members)
* [Training and Testing Split](#Training-and-Testing-Split)
* [Logistic Regression](#Logistic-Regression)
* [Adjust Parameters for Logistic Regression](#Adjust-Parameters-LR)
* [Interpreting Weights](#Interpreting-Weights)
* [Support Vector Machines](#Support-Vector-Machines)
* [Chosen Support Vectors Analysis](#Chosen-Support-Vectors-Analysis)
* [Advantages of each model](#Advantages-of-each-model)
* [References](#References)


# <a name="Team-Members"></a>Team Members
* [Jostein Barry-Straume](https://github.com/josteinstraume)
* [Kevin Cannon](https://github.com/kcannon2)
* [Ernesto Carrera Ruvalcaba](https://github.com/ecarrerasmu)
* [Adam Tschannen](https://github.com/adamtschannen)

> [50 points] Create a logistic regression model and a support vector machine model for the
classification task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). Adjust parameters of the models to make them more accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use. That is, the SGDClassifier is fine to use for optimizing logistic regression and linear support vector machines. For many problems, SGD will be required in order to train the SVM model in a reasonable time frame.

> [10 points] Discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency? Explain in detail.

> [30 points] Use the weights from logistic regression to interpret the importance of different features for the classification task. Explain your interpretation in detail. Why do you think some variables are more important?

> [10 points] Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain. If you used stochastic gradient descent (and therefore did not explicitly solve for support vectors), try sub-sampling your data to train the SVC model— then analyze the support vectors from the sub-sampled dataset.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import statistics as st
import csv as csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from __future__ import print_function
from sklearn.preprocessing import StandardScaler


#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

filepath = "/data/credit-defaults.xls"

# Relative local path to avoid changing filepath constantly
credit = pd.read_excel('../data/credit-defaults.xls', header=1, skiprows=0)


# Rename column(s)
credit = credit.rename(columns={'default payment next month': 'default_next_m', 'PAY_0': 'PAY_1'})


FileNotFoundError: [Errno 2] No such file or directory: '../data/credit-defaults.xls'

In [ ]:
#CREATING NEW VARIABLE TO IMPUTE THE VALUES TO 4, THAT REPRESETNS OTHER
credit['EDUCATION_INP']=credit['EDUCATION']

credit.loc[credit['EDUCATION'] > 4, 'EDUCATION_INP'] = 4
credit.loc[credit['EDUCATION'] == 0, 'EDUCATION_INP'] = 4

In [ ]:
####FOR MARRIAGE
#CREATING NEW VARIABLE TO IMPUTE THE VALUES TO 2, THAT REPRESETNS SINGLE, since the value of 0 is not defined in
#the data dictionary
credit['MARRIAGE_INP']=credit['MARRIAGE']


credit.loc[credit['MARRIAGE'] == 0, 'MARRIAGE_INP'] = 2


In [ ]:
credit.head()

In [ ]:
credit.info()

# <a name="Training-and-Testing-Split"></a>Training and Testing Split

## Stratified K-Folds Cross-Validation

> There is a very apparent class imbalance within the dataset. In order to address this issue, stratified K-Folds cross-validation will be employed to preserve the percentage of samples for each class. In other words, this will prevent the model from simply guessing 'paid duly' for each observation.

In [ ]:
#df = credit
#EC: Creating a another reference in MEMORY, so the changes in df will not be REFLECTED IN CREDIT
df = credit.copy()
from sklearn.model_selection import StratifiedShuffleSplit
if 'default_next_m' in df:
    y = df['default_next_m'].values
    del df['default_next_m']
    del df['ID']
    del df['EDUCATION']
    del df['MARRIAGE']
    X = df.values
num_cv_iterations = 3
num_instances = len(y)
cv_object = StratifiedShuffleSplit(n_splits = num_cv_iterations, 
                            test_size = 0.20, train_size = 0.80, random_state=1)
cv_object.get_n_splits(X, y)
print(cv_object)
for train_index, test_index in cv_object.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
print('# of obs in train set: ' + str(len(y_train)))
print('# of obs in test set: ' + str(len(y_test)))

#train_defaults = sum(y_train == 0)
train_defaults = sum(y_train == 1)
print('# of defaults in train set: ' + str(train_defaults))

proportion_train_defaults = train_defaults / len(y_train)
print('Proportion of defaults in train set: ' + str(proportion_train_defaults))

#test_defaults = sum(y_test == 0)
test_defaults = sum(y_test == 1)
print('# of defaults in test set: ' + str(test_defaults))

proportion_test_defaults = test_defaults / len(y_test)
print('Proportion of defaults in test set: ' + str(proportion_test_defaults))

In [ ]:
#%%time
credit.columns

In [ ]:
df.columns

# <a name="Logistic-Regression"></a>Logistic Regression

> We first run a logistic regression scenario using l2 normal penalty, a cost of 1 and assigning the same weight to the classes in the data.

>Given the fact that the data is unbalanced, we are presenting the metrics derived from the confusion matrix (e.g. Precision, Recall, f1-score). A higher value of the metric "Recall" indicates that the model is predicting most defaults customers correctly.

>Additionally, it is measured the time to fit and train the model to determine the cost of execution.

>The performance of this first configuration is very poor since the "Recall" value for the category 1 (defaulted customers) is 0.

In [ ]:
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from time import time

# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

# now we can use the cv_object that we setup before to iterate through the 
#    different training and testing sets. Each time we will reuse the logisitic regression 
#    object, but it gets trained on different data each time we use it.

iter_num=0
times_rec=[]
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    
    #we count the time in executing the logistic regression
    t0 = time()
    
    # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions
    
    t1=time()
    diff=np.array([t1-t0])
    
    print ("The time it takes to fit and predict is " + str(diff[0]) + "\n")    
    times_rec=np.append(times_rec,diff)
    
    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1
    print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
    ClassReport = mt.classification_report(y_test,y_hat)
    print(ClassReport)
    
    
print("The average time to fit and predict 3 logistic regressions with 80/20 training/test split is: " + str(times_rec.mean()) )

# <a name="Adjust-Parameters-LR"></a>Adjust Parameters for Logistic Regression

> Let's adjust some parameters to see if a more accurate model can be produced.

> By default, the LogisticRegression function within the SciKit Library gives all classes a weight of one. The class_weight parameter will be changed to 'balanced' to see if the accuracy of the models can be improved.

> The results that follow, show an improvement in the performance, the "Recall" value is higher than 0.5, which is deemed adequate for this problem.

In [ ]:
lr_clf = LogisticRegression(penalty='l2', C=1, class_weight='balanced') # get object
#WARNING: THE FIRST WIEGHT WAS 1 
iter_num=0
times_rec=[]
for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
     #we count the time in executing the logistic regression
    t0 = time()
    
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions
    
    t1=time()
    diff=np.array([t1-t0])
    
    print ("The time it takes to fit and predict is " + str(diff[0]) + "\n")    
    times_rec=np.append(times_rec,diff)
    

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    
    print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
    ClassReport = mt.classification_report(y_test,y_hat)
    print(ClassReport)
    
    iter_num+=1
    
    
print("The average time to fit and predict 3 logistic regressions with 80/20 training/test split is: " + str(times_rec.mean()) )

> Now, we will change the normal penalty to be L1 and assigning the same weight to the categories by setting the parameter "class_weight" to "None.

> By default, the LogisticRegression function within the SciKit Learn library uses 'L2' as its 'penalty' parameter. A binary class L2 penalized logistic regression minimizes the following cost function:

In [ ]:
from IPython.display import Image
Image(url='http://scikit-learn.org/stable/_images/math/760c999ccbc78b72d2a91186ba55ce37f0d2cf37.png')

> Now the 'penalty' parameter will be changed to L1 in order to optimize the following problem:

In [ ]:
Image(url='http://scikit-learn.org/stable/_images/math/6a0bcf21baaeb0c2b879ab74fe333c0aab0d6ae6.png')

In [ ]:
lr_clf = LogisticRegression(penalty='l1', C=1.0, class_weight=None) # get object

iter_num=0
times_rec=[]


for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    
    #we count the time in executing the logistic regression
    t0 = time()
    
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions
    
    t1=time()
    diff=np.array([t1-t0])
    
    print ("The time it takes to fit and predict is " + str(diff[0]) + "\n")    
    times_rec=np.append(times_rec,diff)
    

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    
    print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
    ClassReport = mt.classification_report(y_test,y_hat)
    print(ClassReport)
    iter_num+=1
    
print("The average time to fit and predict 3 logistic regressions with 80/20 training/test split is: " + str(times_rec.mean()) )

> The use of L1 penalty did not improve the performance of the model since the value of "Recall" metric decreased. 

> Therefore, the best scenario tested at this point is using L2 normal penalty and setting the class_weight parameter to be “balanced”. The latter uses the values of response variable to automatically adjust weights inversely proportional to class frequencies in the input data


# <a name="Interpreting-Weights"></a>Interpreting Weights

In [ ]:
# and here is an even shorter way of getting the accuracies for each training and test set
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(lr_clf, X, y=y, cv=cv_object) # this also can help with parallelism
print(accuracies)


# here we can change some of the parameters interactively
from ipywidgets import widgets as wd

def lr_explor(cost):
    lr_clf = LogisticRegression(penalty='l2', C=cost, class_weight='balanced') # get object
    accuracies = cross_val_score(lr_clf,X,y=y,cv=cv_object) # this also can help with parallelism
    print(accuracies)

    #WARNING=cost=(0.001,5.0,0.05)
wd.interact(lr_explor,cost=(0.001,5.0,0.05),__manual=True)


# interpret the weights

# iterate over the coefficients
weights = lr_clf.coef_.T # take transpose to make a column vector
variable_names = df.columns
for coef, name in zip(weights,variable_names):
    print(name, 'has weight of', coef[0])

> These weight interpretations are not necessarily interpretable because of the values we had. Very large attribute values could just as easily be assigned a higher weight. Instead, let's normalize the feature values so that all the attributes are on the same dynamic range. Once we normalize the attributes, the weights should have magnitudes that reflect their predictive power in the logistic regression model.

In [ ]:
from sklearn.preprocessing import StandardScaler

# we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)
# to Make things easier, let's start by just using whatever was last stored in the variables:
##    X_train , y_train , X_test, y_test (they were set in a for loop above)

# scale attributes by the training set
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find scalings for each column that make this zero mean and unit std
# the line of code above only looks at training data to get mean and std and we can use it 
# to transform new feature data

X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

# train the model just as before
lr_clf = LogisticRegression(penalty='l2', C=1, class_weight='balanced') # get object, the 'C' value is less (can you guess why??)
lr_clf.fit(X_train_scaled,y_train)  # train object

y_hat = lr_clf.predict(X_test_scaled) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf )

# sort these attributes and spit them out
zip_vars = zip(lr_clf.coef_.T,df.columns) # combine attributes
zip_vars = sorted(zip_vars)
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0]) # now print them out

In [ ]:
# now let's make a pandas Series with the names and values, and plot them
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


weights = pd.Series(lr_clf.coef_[0],index=df.columns)
weights.plot(kind='bar')
plt.show()

In [ ]:
from pydoc import help
from scipy.stats.stats import pearsonr

#correlation with in like variables 
bill = pearsonr(credit.BILL_AMT1, credit.BILL_AMT2)
pay = pearsonr(credit.PAY_AMT1, credit.PAY_AMT2)


print('bill amount correlation:', bill)
print('pay amount correlation:', pay)


> For more improvement and guarding against overfitting: At this point it make sense to remove variables highly related to one another and irrelevant ones and keep going with the weights analysis.

> Initally only a handful of the explanatory variables seem to be worth keeping in the model. In particular, PAY_1, BILL_AMT1, BILL_AMT2, PAY_AMT1 and PAY_AMT2 have the greatest absolute weights. After taking a look at the correlations between BILL_AMT1 and BILL_AMT2 , with a correlation above .95, we can delete BILL_AMT2 to prevent over fitting. Like wise, we can keep both PAY_AMT1 and PAY_AMT2 becasue of the low correlation of .285.




In [ ]:
#correlation with response variable
gender = pearsonr(credit.SEX, credit.default_next_m)
Marrige = pearsonr(credit.MARRIAGE_INP, credit.default_next_m)
age = pearsonr(credit.AGE, credit.default_next_m)
education = pearsonr(credit.EDUCATION_INP, credit.default_next_m)
#billD = pearsonr(credit.BILL_AMT1, credit.default_next_m)
#payD1 = pearsonr(credit.PAY_AMT1, credit.default_next_m)
#payD2 = pearsonr(credit.PAY_AMT2, credit.default_next_m)
print('age default correlation:', age)
print('marrige default correlation:', Marrige)
print('gender default correlation:', gender)
print('education default correlation:', education)
#print('bill default correlation:', billD)
#print('pay1 default correlation:', payD1)
#print('pay2 default correlation:', payD2)




> Furthermore, Education, Marriage, Age and Sex appears to have only marginal significance in the model, but it could be argued to keep them in the model if all redundant PAY_X, BILL_AMTX, and PAY_AMTX variables are being removed. The argument to keep these variables in the model would be to help classify people before they have a payment or bill history with the company. For instance, a company could offer customer different products or marketing campaigns based on categories they would fall in if we used the attributes in our model.

> After taking a further look at these "profiling" features, if seems reasonable to keep the top two features with the highest weights, Marriage and Education, in our model.


In [ ]:
from sklearn.preprocessing import StandardScaler
# we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)

from sklearn.pipeline import Pipeline
# you can apply the StandardScaler function inside of the cross-validation loop 
#  but this requires the use of PipeLines in scikit. 
#  A pipeline can apply feature pre-processing and data fitting in one compact notation
#  Here is an example!

std_scl = StandardScaler()
lr_clf = LogisticRegression(penalty='l2', C=1, class_weight='balanced') 

# create the pipline
piped_object = Pipeline([('scale', std_scl),  # do this
                         ('logit_model', lr_clf)]) # and then do this

weights = []
# run the pipline cross validated
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    piped_object.fit(X[train_indices],y[train_indices])  # train object
    # it is a little odd getting trained objects from a  pipeline:
    weights.append(piped_object.named_steps['logit_model'].coef_[0])
    

weights = np.array(weights)

In [ ]:
#from plotly import __version__
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#plotly.offline.init_notebook_mode() # run at the start of every notebook

import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
print("Plotly version: " + plotly.__version__)           # version 1.9.x required
plotly.offline.init_notebook_mode() # run at the start of every notebook

error_y=dict(
            type='data',
            array=np.std(weights,axis=0),
            visible=True
        )

graph1 = {'x': df.columns,
          'y': np.mean(weights,axis=0),
    'error_y':error_y,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)

In [ ]:
Xnew = df[['PAY_1','BILL_AMT1','PAY_AMT1','PAY_AMT2','EDUCATION_INP','MARRIAGE_INP']].values

weights = []
# run the pipline corssvalidated
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(Xnew,y)):
    piped_object.fit(Xnew[train_indices],y[train_indices])  # train object
    weights.append(piped_object.named_steps['logit_model'].coef_[0])
    
weights = np.array(weights)

error_y=dict(
            type='data',
            array=np.std(weights,axis=0),
            visible=True
        )

graph1 = {'x': ['PAY_1','BILL_AMT1','PAY_AMT1','PAY_AMT2','EDUCATION_INP','MARRIAGE_INP'],
          'y': np.mean(weights,axis=0),
    'error_y':error_y,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)

> At first glance one might think each bill and payment period should have equal weight within the logistic regression model. However, clearly the first bill and payment period have the most significance upon viewing the above plots. Perhaps the greater weight of the first period compared to the other five can be explained by the recency of the period. 

> Simply put, if a customer fails to pay their credit bill in the first opportunity to pay, they are marked as having defaulted in the dataset. In other words, if a customer is likely to default on their bills, then the first chance for said customer is the first period.

> If a customer has paid duly for periods 1-5, then they are likely to also pay their credit bill on the 6th go around, thus exhibiting they are fiscally sound as a customer.


# <a name="Support-Vector-Machines"></a>Support Vector Machines

In [ ]:
from IPython.display import Image
# Big Data, in all its glory:
Image(url='http://flowingdata.com/wp-content/uploads/2014/11/Big-Data-620x465.jpg')

>In this section, we will explore different configurations of SVM to derive a model to classify defaulted and non-defaulted customers.

> As we did for the logistic regression, we are presenting the metrics derived from the confusion matrix (e.g. Precision, Recall, f1-score) since our data in unbalanced. A higher value of the metric "Recall" indicates that the model is predicting most defaults customers correctly. Lastly, the time to fit and predict is also measured.

>The first configuration tested uses a "linear" kernel with a cost of 0.5 (C-parameter) a degree of 3 and gamma set to auto.

>According to sklearn documentation:

>Intuitively, the gamma parameter defines how far the influence of a single training example reaches, with low values meaning ‘far’ and high values meaning ‘close’. The gamma parameters can be seen as the inverse of the radius of influence of samples selected by the model as support vectors.

>The C parameter trades off misclassification of training examples against simplicity of the decision surface. A low C makes the decision surface smooth, while a high C aims at classifying all training examples correctly by giving the model freedom to select more samples as support vectors

In [ ]:
# okay, so run through the cross validation loop and set the training and testing variable for one single iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test)

In [ ]:
# lets investigate SVMs on the data and play with the parameters and kernels - CHANGING THE KERNEL TO BE LINEAR
from sklearn.svm import SVC

# train the model just as before
#warning
#svm_clf = SVC(C=0.5, kernel='rbf', degree=3, gamma='auto') # get object
svm_clf = SVC(C=0.5, kernel='linear', degree=3, gamma='auto') # get object

#we count the time in executing the logistic regression
t0 = time()
svm_clf.fit(X_train_scaled, y_train)  # train object

y_hat = svm_clf.predict(X_test_scaled) # get test set precitions

t1=time()
diff=t1-t0
    
print ("The time it takes to fit and predict is " + str(diff) + "\n")  

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf)


print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
ClassReport = mt.classification_report(y_test,y_hat)
print(ClassReport)

>The value of recall exhibited above is of 0.23 which is considerably lower than the best scenario of the logistic regression.

In [ ]:
# look at the support vectors
print(svm_clf.support_vectors_.shape)
print(svm_clf.support_.shape)
print(svm_clf.n_support_ )

> Now, keeping other parameters equal, we use the Radial Basis Function (RBF) kernel.

In [ ]:
# lets investigate SVMs on the data and play with the parameters and kernels - CHANGING THE KERNEL TO BE RBF
from sklearn.svm import SVC

# train the model just as before
#warning
svm_clf = SVC(C=0.5, kernel='rbf', degree=3, gamma='auto') # get object
#svm_clf = SVC(C=0.5, kernel='linear', degree=3, gamma='auto') # get object

#we count the time in executing the logistic regression
t0 = time()
svm_clf.fit(X_train_scaled, y_train)  # train object

y_hat = svm_clf.predict(X_test_scaled) # get test set precitions

t1=time()
diff=t1-t0
    
print ("The time it takes to fit and predict is " + str(diff) + "\n")  

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf)


print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
ClassReport = mt.classification_report(y_test,y_hat)
print(ClassReport)

> The results above show a higher value of the "Recall" metric which in this case indicates an improvement.

In [ ]:
# look at the support vectors
print(svm_clf.support_vectors_.shape)
print(svm_clf.support_.shape)
print(svm_clf.n_support_ )

>Following with the configuration presented above, now we change the parameter C equals to 2

In [ ]:
# lets investigate SVMs on the data and play with the parameters and kernels - CHANGING THE KERNEL TO BE RBF
#AND CHANGING THE COST TO BE 2
from sklearn.svm import SVC

# train the model just as before
#warning
svm_clf = SVC(C=2, kernel='rbf', degree=3, gamma='auto') # get object
#svm_clf = SVC(C=0.5, kernel='linear', degree=3, gamma='auto') # get object

#we count the time in executing the logistic regression
t0 = time()
svm_clf.fit(X_train_scaled, y_train)  # train object

y_hat = svm_clf.predict(X_test_scaled) # get test set precitions

t1=time()
diff=t1-t0
    
print ("The time it takes to fit and predict is " + str(diff) + "\n")  

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf)


print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
ClassReport = mt.classification_report(y_test,y_hat)
print(ClassReport)

>The results above show a higher value of the "Recall" metric which in this case indicates an improvement.

In [ ]:
# look at the support vectors
print(svm_clf.support_vectors_.shape)
print(svm_clf.support_.shape)
print(svm_clf.n_support_ )

>Using the latest configuration, now we change the parameter gamma to be 1e1. This change does not improve the performance of the model as the results below exhibit since the value of the recall decreases.

In [ ]:
# lets investigate SVMs on the data and play with the parameters and kernels - CHANGING THE KERNEL TO BE RBF
#AND CHANGING THE WEIGHT TO BE 2 AND GAMMA=1e1
from sklearn.svm import SVC

# train the model just as before
#warning
svm_clf = SVC(C=2, kernel='rbf', degree=3, gamma=1e1) # get object
#svm_clf = SVC(C=0.5, kernel='linear', degree=3, gamma='auto') # get object

#we count the time in executing the logistic regression
t0 = time()
svm_clf.fit(X_train_scaled, y_train)  # train object

y_hat = svm_clf.predict(X_test_scaled) # get test set precitions

t1=time()
diff=t1-t0
    
print ("The time it takes to fit and predict is " + str(diff) + "\n")  

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf)


print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
ClassReport = mt.classification_report(y_test,y_hat)
print(ClassReport)

In [ ]:
# look at the support vectors
print(svm_clf.support_vectors_.shape)
print(svm_clf.support_.shape)
print(svm_clf.n_support_ )

>The best configuration identified at this point for SVM uses the "rbf" kernel with a cost of 2, degree of 3 and the gamma parameter equal to 2. To validate the performance of this configuration, to avoid overfitting we execute a cross-validation with 3 folds. The results in the "recall" metric are very similar, as exhibited below.

In [ ]:
############### CV FOR SVM TO VALIDATE OVERFITTING #######################

# we are using the best scenario to run the SVM AFTER changing some of the parameters
#lr_clf = LogisticRegression(penalty='l1', C=1.0, class_weight=None) # get object
svm_clf = SVC(C=2, kernel='rbf', degree=3, gamma='auto')


iter_num=0
times_rec=[]


for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    #### SCALING THE  FOLD
    #OBTAINING THE TUNING PARAMETERS FOR EACHV ARIABLE IN THE TRAINING SAMPLE
    scl_obj.fit(X_train) # find scalings for each column that make this zero mean and unit std
    # the line of code above only looks at training data to get mean and std and we can use it 
    # to transform new feature data
    
    
    
    X_train_scaled = scl_obj.transform(X_train) # apply to training
    X_test_scaled = scl_obj.transform(X_test)
    
    
    #we count the time in executing the logistic regression
    t0 = time()
    
    svm_clf.fit(X_train_scaled,y_train)  # train object
    y_hat = svm_clf.predict(X_test_scaled) # get test set precitions
    
    t1=time()
    diff=np.array([t1-t0])
    
    print ("The time it takes to fit and predict is " + str(diff[0]) + "\n")    
    times_rec=np.append(times_rec,diff)
    

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    
    print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
    ClassReport = mt.classification_report(y_test,y_hat)
    print(ClassReport)
    iter_num+=1
    
print("The average time to fit and predict 3 SVM with 80/20 training/test split is: " + str(times_rec.mean()) )



# <a name="Chosen-Support-Vectors-Analysis"></a>Chosen Support Vectors Analysis

> In this section, we take a look at the chosen support vectors (with the best scenario obtained in the prior section) for the classification task in order to identify any insight into the data.

In [ ]:
################ FOR THE CONFIGURATION SELECTED WE NEED TO RE-RUN THE CODE TO DO THE ANALYSIS OF
###### CHOSEN SUPPORT VECTORS

# lets investigate SVMs on the data and play with the parameters and kernels - CHANGING THE KERNEL TO BE RBF
#AND CHANGING THE COST TO BE 2
from sklearn.svm import SVC

# train the model just as before
#warning
svm_clf = SVC(C=2, kernel='rbf', degree=3, gamma='auto') # get object
#svm_clf = SVC(C=0.5, kernel='linear', degree=3, gamma='auto') # get object

#we count the time in executing the logistic regression
t0 = time()
svm_clf.fit(X_train_scaled, y_train)  # train object

y_hat = svm_clf.predict(X_test_scaled) # get test set precitions

t1=time()
diff=t1-t0
    
print ("The time it takes to fit and predict is " + str(diff) + "\n")  

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf)


print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
ClassReport = mt.classification_report(y_test,y_hat)
print(ClassReport)



In [ ]:
# look at the support vectors
print(svm_clf.support_vectors_.shape)
print(svm_clf.support_.shape)
print(svm_clf.n_support_ )

In [ ]:
# Now let's do some different analysis with the SVM and look at the instances that were chosen as support vectors

# now lets look at the support for the vectors and see if we they are indicative of anything
# grabe the rows that were selected as support vectors (these are usually instances that are hard to classify)

# make a dataframe of the training data
df_tested_on = df.iloc[train_indices] # saved from above, the indices chosen for training
# now get the support vectors from the trained model
df_support = df_tested_on.iloc[svm_clf.support_,:]

df_support['default_next_m'] = y[svm_clf.support_] # add back in the 'Survived' Column to the pandas dataframe
df['default_next_m'] = y # also add it back in for the original data
df_support.info()

> The following charts exhibit the KDE broke down by defaulted and non-defaulted customers. The charts on the left show the chosen support vectors and the charts on the right present the distribution of the variable. At the end, Support Vectors are simply the coordinates of individual observation. A Support Vector is a frontier which best segregates the two classes (hyper-plane/ line).

>For example, the variable "Pay_1", the value of "0" shows a smaller gap between defaulted and non-defaulted, in other words, we are seeing the observations that are deemed as an error and used by the SVM to build the frontier.

>The distributions for the variable "Marriage_INP" between the chosen support vectors and the real data looks very similar, that is telling us that there were not many instances identified as "errors" and therefore selected as Support Vectors based on the values for this variable.


In [ ]:
# now lets see the statistics of these attributes
from pandas.tools.plotting import boxplot

# group the original data and the support vectors
df_grouped_support = df_support.groupby(['default_next_m'])
df_grouped = df.groupby(['default_next_m'])

# plot KDE of Different variables
vars_to_plot = ['PAY_1','BILL_AMT1','PAY_AMT1','EDUCATION_INP','MARRIAGE_INP']

for v in vars_to_plot:
    plt.figure(figsize=(10,4))
    # plot support vector stats
    plt.subplot(1,2,1)
    ax = df_grouped_support[v].plot.kde() 
    plt.legend(['Default','Paid Duly'])
    plt.title(v+' (Instances chosen as Support Vectors)')
    
    # plot original distributions
    plt.subplot(1,2,2)
    ax = df_grouped[v].plot.kde() 
    plt.legend(['Default','Paid Duly'])
    plt.title(v+' (Original)')

> By plotting the original data density statistics next to the statistics for the support vectors, we can look at the separation between the distributions of the defaults and those that paid duly. Generally, the separations between the original data will be larger than the separations from the support vector data, since the support vectors use points on the decision boundary to create the dividing line for classification. Otherwise, the points that are used are errors. Since the support vectors use edge points, the kernel density estimation lines should appear close together. And, the data for the PAY_1 and BILL_AMT1 variables do indeed have smaller separations between the distributions for the support vectors. This development is consistent with the weights calculated previously. Since PAY_1 and BILL_AMT1 are so much more significant to the model predictions than any of the other variables, it makes sense that these two variables make up many of the support vectors. Since more data points using those two variables are used from the original data from the support vectors, the distribution between two is a closer match, as seen in the charts.

> However, the three remaining variables – PAY_AMT1, EDUCATION_INP, and MARRIAGE_INP - have similar or even slightly larger separations between the distributions. Since these variables are not weighted as heavily as the two primary prediction variables, the data diverges slightly between the original and support vector plots. The separation can vary depending on the size of the margins. These values could have error values that negatively impact the KDE lines that are drawn between the original and support vector data, leading to the slightly increased separations. 

In [ ]:
# Joint Plot
# Source:
# http://seaborn.pydata.org/generated/seaborn.jointplot.html
import seaborn as sns

# Original dataset
g = sns.jointplot("MARRIAGE_INP", "PAY_1", data=df, kind="kde", space=0, color="g")

# Support Vector
g = sns.jointplot("MARRIAGE_INP", "PAY_1", data=df_support, kind="kde", space=0, color="g")

> Looking at the bivariate distribution of the two variables, the correlation between the PAY_1 variable and the MARRIAGE_INP improves in the support vector data over the original data. As seen in the plots, the density around all combinations of points improve and is most noticeable around the central points.

In [ ]:
# Joint Plot
# Source:
# http://seaborn.pydata.org/generated/seaborn.jointplot.html
import seaborn as sns;
x = df_support['MARRIAGE_INP']
y = df_support['PAY_1']
g = (sns.jointplot(x, y, kind="hex", stat_func=None).set_axis_labels("Marriage", "Pay_1"))

> Taking another look at the PAY_1 and MARRIAGE_INP variables for the support vector data, the density improvement around the central values can be seen in the very dark points in the joint histogram using hexagonal bins.

In [ ]:
g = sns.jointplot("BILL_AMT1", "PAY_AMT1", data=df, kind="hex")

># <a name="Advantages-of-each-model"></a>Advantages of each model

> For this classification problem, we recommend to use the logistic regression based upon the following:
    
       >1)The classification metrics of "Recall" and "F1-score" are higher for the logistic regression. The data is unbalanced and therefore, the "Recall" is a better metric to measure the performance of the model since a high value means that it is predicting more default customers correctly. 
       
       >2) The time of execution is considerably higher for SVM. On average, the time taken to fit and predict the model is around 1 second using logistic regression whereas the SVM takes around 30 seconds. 
       
       >3) The weights of the logistic regression provide a method to derive the importance of each variable. The bank running this model would be interested to know what variables are key to identify defaults to create segments that allow them to offer customer different products or marketing campaigns.

# <a name="References"></a>References

* https://github.com/eclarson/DataMiningNotebooks/blob/master/04.%20Logits%20and%20SVM.ipynb
* https://github.com/jakemdrew/EducationDataNC/blob/master/Graduation%20Rates%20v2.ipynb
* http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
* http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
* https://stackoverflow.com/questions/29438265/stratified-train-test-split-in-scikit-learn
* http://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/
* https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
